## **API's URL**

In [ ]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'
api_url = 'https://api.adviceslip.com/advice'

## **Extract**

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('sdw2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None
users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

## **Transform**

Utilize uma API para gerar uma mensagem personalizada para cada usuário.

In [ ]:
def generate_ai_news(user):
  response = requests.get(f'{api_url}?user={user}')
  if response.status_code == 200:
    return response.json()
  else:
    print(f'Erro ao obter o conselho para o usuário {user}.')


for user in users:
  mensagem = generate_ai_news(user)
  print("{}:{}".format(user['name'],mensagem['slip']['advice']))
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
      "description": mensagem['slip']['advice']
  })

## **Load**

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json = user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")
